In [ ]:
from IPython.display import clear_output, Image, display, HTML

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1200px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [1]:
import tensorflow as tf
import numpy as np

In [ ]:
a = [1., 2., 3.]
print a

b = [[1., 2., 3.], [4., 5., 6.]]
print b

c = [[[1., 2., 3.]], [[7., 8., 9.]]]
print c

In [ ]:
node1 = tf.constant(3.0, dtype=tf.float32)
node2 = tf.constant(4.0) # implicitly tf.float32
print (node1, node2)

In [ ]:
sess = tf.Session()

In [ ]:
print sess.run([node1, node2])

In [ ]:
node3 = tf.add(node1, node2)
print node3
print sess.run(node3)

In [ ]:
show_graph(tf.get_default_graph().as_graph_def())

In [ ]:
a = tf.placeholder(tf.float32)
b = tf.placeholder(tf.float32)
node4 = a + b # shortcut for tf.add(a,b)

In [ ]:
print sess.run(node4, {a: 3, b: 4.5}) # pass in values
print sess.run(node4, {a: [1, 2], b: [3, 4]}) # pass in arrays

In [ ]:
node5 = node4 * 3
print sess.run(node5, { a: 3, b: 4 })

In [ ]:
W = tf.Variable([.3], dtype=tf.float32)
b = tf.Variable([-.3], dtype=tf.float32)
x = tf.placeholder(tf.float32) # you only specify the type, not the dimension

In [ ]:
linear_model = W * x + b

In [ ]:
init = tf.global_variables_initializer() # need to explicitly initialize variables
sess.run(init)

In [ ]:
print sess.run(linear_model, { x: [1, 2, 3, 4]})

In [ ]:
y = tf.placeholder(tf.float32)
squared_err = tf.square(linear_model - y)
loss = tf.reduce_sum(squared_err) # err = (y^ - y)^2
print sess.run(loss, {x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

In [ ]:
Wbest = tf.assign(W, [-1.]) # assign W to [-1.], changes are made for relevant models
bbest = tf.assign(b, [1.])
sess.run([Wbest, bbest])
print sess.run(loss, { x: [1,2,3,4], y:[0,-1,-2,-3]})

In [ ]:
opt = tf.train.GradientDescentOptimizer(0.01)
train = opt.minimize(loss)

In [ ]:
sess.run(init) # reinitialize model variables using global_variables_initializer, W and b are set back to 0.3, -0.3
iters = 1000
for i in range(iters):
    sess.run(train, { x: [1, 2, 3, 4], y: [0, -1, -2, -3]})

print sess.run([W, b])

In [ ]:
# High level API
features = [tf.contrib.layers.real_valued_column("x", dimension=1)]

estimator = tf.contrib.learn.LinearRegressor(feature_columns=features)

x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])

input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, batch_size=4, num_epochs=1000)
eval_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_eval}, y_eval, batch_size=4, num_epochs=1000)

estimator.fit(input_fn=input_fn, steps=1000) # call session.run

In [ ]:
train_loss = estimator.evaluate(input_fn = input_fn)
eval_loss = estimator.evaluate(input_fn = eval_fn)
print "train loss: %r" % train_loss
print "eval loss: %r" % eval_loss

In [18]:
# writing custom models, implement linear regression using low level API
# we need to pass a function that takes the following parameters:
# - features
# - labels
# - mode
# and returns the following things to the Estimator class:
# - predictions, e.g. y=wx+b
# - loss
# - training steps: (what optimizer you're using and how the iteration number "global_step" is being updated)
def model(features, labels, mode):
    # define the parameters for this model
    W = tf.get_variable("W", [1], dtype=tf.float64) # shape is [1]
    b = tf.get_variable("b", [1], dtype=tf.float64) # these seem to be globally stored
    # - predictions
    y = W*features['x'] + b
    # - loss
    loss = tf.reduce_sum(tf.square(y - labels))
    # training steps
    global_step = tf.train.get_global_step() # the iteration number seems to be counted globally... maybe for paralell stuff?
    optimizer = tf.train.GradientDescentOptimizer(0.01) # this must be the step size
    train = tf.group(optimizer.minimize(loss), tf.assign_add(global_step, 1)) # group these two operations (I guess make sure they both finish? why?)
    
    return tf.contrib.learn.ModelFnOps(mode=mode, predictions=y, loss=loss, train_op=train)

In [ ]:
estimator = tf.contrib.learn.Estimator(model_fn = model) # inject our model class/function
x_train = np.array([1., 2., 3., 4.])
y_train = np.array([0., -1., -2., -3.])
x_eval = np.array([2., 5., 8., 1.])
y_eval = np.array([-1.01, -4.1, -7, 0.])
input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_train}, y_train, 4, num_epochs=1000)
eval_input_fn = tf.contrib.learn.io.numpy_input_fn({"x": x_eval}, y_eval)
# we have 2 things here: the model with how to calculate loss and gradients and the learning data and loop parameters
estimator.fit(input_fn = input_fn) # I don't understand why you need steps and batchsize+epochs

In [25]:
# Here we evaluate how well our model did. 
train_loss = estimator.evaluate(input_fn=input_fn)
eval_loss = estimator.evaluate(input_fn=eval_input_fn)
print("train loss: %r"% train_loss)
print("eval loss: %r"% eval_loss)

INFO:tensorflow:Starting evaluation at 2017-07-30-22:46:30
INFO:tensorflow:Restoring parameters from /tmp/tmpeiRaVn/model.ckpt-2
INFO:tensorflow:Finished evaluation at 2017-07-30-22:46:30
INFO:tensorflow:Saving dict for global step 2: global_step = 2, loss = 167.552


NameError: name 'eval_input_fn' is not defined